In [1]:
from pyomo.core import *
from pyomo.opt import SolverFactory, SolverManagerFactory
import pyomo.environ
import pandas as pd
import numpy as np
from itertools import compress
import xarray as xr

In [2]:
from IPython.core.display import HTML
HTML("<style>.container { width:80% !important; }</style>")

In [189]:
class InputData:
    
    #----------------------------------------------------------------------
    
    def __init__(self, ExcelPath, demands='Demand data',solar='Solar data', tech='Technology', gen='General'):
        self.path = ExcelPath
        self.DemandSheet = demands
        self.SolarSheet = solar
        self.TechSheet = tech
        self.GeneralSheet = gen
        
        self.TechOutputs = None
        self.Technologies = None
        self.DemandData = None
        self.StorageData = None
        self.numberofhubs = None
        self.numberofdemands = None
        
        self.Initialize()
        
    def Initialize(self):
        self.Hubs()
        self.Demanddata()
        self.TechParameters()
        self.TechOutput()
        self.Storage()        
        
    def TechParameters(self):
        Technologies=pd.read_excel(self.path,sheetname=self.TechSheet, skiprows=1, index_col=0, skip_footer=38) #technology characteristics
        Technologies=Technologies.dropna(axis=1, how='all') #technology characteristics
        Technologies=Technologies.fillna(0) #technology characteristics
        dd={}
        for i in range(self.numberofhubs):
            dd[i]=Technologies
        self.Technologies=pd.Panel(dd)
    
    def TechOutput(self):
        TechOutputs=pd.read_excel(self.path,sheetname=self.TechSheet, skiprows=15, index_col=0, skip_footer=28) #Output matrix
        TechOutputs=TechOutputs.dropna(axis=0,how='all')  #Output matrix
        TechOutputs=TechOutputs.dropna(axis=1,how='all')  #Output matrix
        self.TechOutputs=TechOutputs
        
    def Demanddata(self):
        DemandDatas=pd.read_excel(self.path,sheetname=self.DemandSheet, header=None, skiprows=0)
        self.numberofdemands = DemandDatas[1][0]
        dd={}
        for i in range(self.numberofhubs):
            dd[i] = DemandDatas.loc[3:,(i+1)*self.numberofdemands-(self.numberofdemands-1):(i+1)*self.numberofdemands] #every i-th is new hub
        self.DemandData=pd.Panel(dd)
        
    def Storage(self):
        Storage=pd.read_excel(self.path,sheetname=self.TechSheet, skiprows=40, index_col=0, skip_footer=0, header = 0) #
        Storage=Storage.dropna(axis=1, how='all')
        Storage=Storage.fillna(0)
        self.StorageData=Storage
        
    def Hubs(self):
        number=pd.read_excel(self.path,sheetname=self.GeneralSheet, skiprows=17, index_col=0, skip_footer=0) #Output matrix
        number=number.dropna(axis=1,how='all')  #Output matrix
        number=number.iloc[0][0]
        self.numberofhubs=number
        
    
    
    
    #----------------------------------------------------------------------

    def Dict1D(self,dictVar,dataframe):
        for i,vali in enumerate(dataframe.index):
            dictVar[i+1]=round(dataframe.iloc[i][1],4)
        return dictVar

    def Dict1D_val_index(self,dictVar,dataframe):
        for i,vali in enumerate(dataframe.index):
            dictVar[vali]=round(dataframe.iloc[i][1],4)
        return dictVar

    def DictND(self,dictVar,dataframe):
        for i,vali in enumerate(dataframe.index):
            for j,valj in enumerate(dataframe.columns):
                #dictVar[vali,valj]=dataframe.iloc[i][j+1]
                dictVar[vali,valj]=dataframe.loc[vali][valj]
        return dictVar
    
    def DictPanel(self, dictVar,panel):
        for x,valx in enumerate(panel.items):
            for i, vali in enumerate(panel[valx].dropna(axis=0, how ='all').index):
                for j, valj in enumerate(panel[valx].dropna(axis=1, how ='all').columns):
                    #panel[valx] = panel[valx].dropna(axis=1)
                    #panel[valx] = panel[valx].dropna(axis=0,how='all')
                    dictVar[x+1,j+1, i+1] = panel[valx].loc[vali][valj]
        return dictVar
    
    
    #----------------------------------------------------------------------
    
    def Demands(self):
        dummy={}
        for i in range(self.DemandData.shape[0]):
            dummy[i]=self.DemandData[i].transpose()
        
        Demand=pd.Panel(dummy)
        loads_init={}
        loads_init=self.DictPanel(loads_init,Demand)
        return loads_init
    
    
    
    
    #----------------------------------------------------------------------
    
    def SolarData(self):
        SolarData=pd.read_excel(self.path,sheetname=self.SolarSheet)
        SolarData.columns=[1]
        solar_init={}
        solar_init=self.Dict1D(solar_init,SolarData)
        return solar_init
    
    
    
    
    #----------------------------------------------------------------------
    
    def CHP_list(self):
        Dispatch_tech=pd.DataFrame(self.TechOutputs.sum(0)) #find dispatch tech (CHP)
        CHP_setlist=[]
        for n,val in enumerate(Dispatch_tech[0]):
            if val>1:
                CHP_setlist.append(n+2) #first is electricity +1 since it starts at 0
        return CHP_setlist
    
    
    
    
    #----------------------------------------------------------------------
    
    def Roof_tech(self):
        Roof_techset=[]
        for n,val in enumerate(self.Technologies[0].loc["Area (m2)"]): #take only 0th item
            if val>0:
                Roof_techset.append(n+2) #first is electricity +1 since it starts at 0
                
        return Roof_techset
    
    
    
    
    #----------------------------------------------------------------------
        
    def cMatrix(self):
        
        #Based on the + - values, prepare data for generating coupling matrix        
        TechOutputs2=self.TechOutputs.multiply(np.array(self.Technologies[0].loc['Efficiency (%)']))
        TechOutputs2.loc[TechOutputs2.index!='Electricity']=TechOutputs2.loc[TechOutputs2.index!='Electricity'].multiply(np.array(self.Technologies[0].loc['Output ratio'].fillna(value=1).replace(0,1)))
        TechOutputs2[TechOutputs2<0]=-1

        addGrid=np.zeros(len(self.DemandData[0].dropna(axis=1, how ='all').columns),)
        addGrid[0]=1 #add electricity to coupling matrix
        Grid=pd.DataFrame(addGrid,columns=["Grid"],index=self.DemandData[0].dropna(axis=1, how ='all').columns).transpose()
        
        Cmatrix=TechOutputs2.transpose()
        Cmatrix.columns = list(range(1,len(Cmatrix.columns)+1))
        Cmatrix=pd.concat([Grid,Cmatrix])
        Cmatrix.index=list(range(1,len(TechOutputs2.columns)+2))
        Cmatrix.columns=list(range(1,len(TechOutputs2.index)+1))
        cMatrixDict={}
        cMatrixDict=self.DictND(cMatrixDict,Cmatrix)
        
        return cMatrixDict
    
    
    
    
    #----------------------------------------------------------------------
    
    def PartLoad(self):
        PartLoad=self.Technologies[0].loc["MinLoad (%)",]/100

        partload=self.TechOutputs.iloc[0:1].mul(list(PartLoad),axis=1)
        partload=pd.concat([partload,self.TechOutputs.iloc[1:].mul(list(PartLoad),axis=1)], axis=0)
        partload=partload.abs()
        partload=partload.transpose()
        partload.index=list(range(1+1,len(self.TechOutputs.columns)+2))
        partload.columns=list(range(1,len(self.TechOutputs.index)+1))
        SolartechsSets=list(compress(list(range(1+1,len(self.Technologies[0].columns)+2)), list(self.Technologies[0].loc["Area (m2)"]>0)))

        for i in SolartechsSets:
            partload.drop(i, inplace=True)

        PartloadInput={}
        PartloadInput=self.DictND(PartloadInput,partload)
        
        return PartloadInput
    
    
    
    
    #----------------------------------------------------------------------
    
    def MaxCapacityALL(self):
        maxCap={}
        for n in range(self.numberofhubs):
            MaxCap=pd.DataFrame(self.Technologies[n].loc["Maximum Capacity",])
            MaxCap.index=list(range(2,len(self.Technologies[n].loc["MinLoad (%)",].index)+2))
            maxCap[n]= MaxCap

            SolartechsSets=list(compress(list(range(1+1,len(self.Technologies[n].columns)+2)), list(self.Technologies[n].loc["Area (m2)"]>0)))
            #maxCap[n] = maxCap[n].drop(SolartechsSets)
            for x in SolartechsSets:
                maxCap[n].iloc[x-2] = list(data.Technologies[n].loc["Area (m2)"])[x-2]
            if self.numberofhubs>2:
                for k in maxCap[n].index:
                    if str(n+1) not in list(self.Technologies[n].loc["Hubs"][k-2]):
                        maxCap[n].loc[k] = 0
            elif self.numberofhubs==2:
                for k in maxCap[n].index:
                    if isinstance(data.Technologies[n].loc["Hubs"][k-2],float):
                        if int(n+1) != int(self.Technologies[n].loc["Hubs"][k-2]):
                            maxCap[n].loc[k] = 0
        
        CapacitiesPanel=pd.Panel(maxCap)
        
        Capacities = CapacitiesPanel.to_frame()
        Capacities.reset_index(inplace = True)
        Capacities.index = Capacities['major']
        del Capacities['major']
        del Capacities['minor']
        Capacities.columns = [int(x)+1 for x in Capacities.columns]
        del Capacities.index.name
        Capacities = Capacities.transpose()
        
        maximalcapacities={}
        maximalcapacities = self.DictND(maximalcapacities, Capacities)
        return maximalcapacities
    
    def MaxCapacity(self):
        maxCap={}
        for n in range(self.numberofhubs):
            MaxCap=pd.DataFrame(self.Technologies[n].loc["Maximum Capacity",])
            MaxCap.index=list(range(2,len(self.Technologies[n].loc["MinLoad (%)",].index)+2))
            maxCap[n]= MaxCap

            SolartechsSets=list(compress(list(range(1+1,len(self.Technologies[n].columns)+2)), list(self.Technologies[n].loc["Area (m2)"]>0)))
            maxCap[n] = maxCap[n].drop(SolartechsSets)
            if self.numberofhubs>2:
                for k in maxCap[n].index:
                    if str(n+1) not in list(self.Technologies[n].loc["Hubs"][k-2]):
                        maxCap[n].loc[k] = 0
            elif self.numberofhubs==2:
                for k in maxCap[n].index:
                    if isinstance(data.Technologies[n].loc["Hubs"][k-2],float):
                        if int(n+1) != int(self.Technologies[n].loc["Hubs"][k-2]):
                            maxCap[n].loc[k] = 0
                        
        CapacitiesPanel=pd.Panel(maxCap)
        
        Capacities = CapacitiesPanel.to_frame()
        Capacities.reset_index(inplace = True)
        Capacities.index = Capacities['major']
        del Capacities['major']
        del Capacities['minor']
        Capacities.columns = [int(x)+1 for x in Capacities.columns]
        del Capacities.index.name
        Capacities = Capacities.transpose()
        
        maximalcapacities={}
        maximalcapacities = self.DictND(maximalcapacities, Capacities)
        return maximalcapacities
     
    
    
    
    #----------------------------------------------------------------------
    
    def SolarSet(self):
        return list(compress(list(range(1+1,len(self.Technologies[0].columns)+2)), list(self.Technologies[0].loc["Area (m2)"]>0)))
    
    
    def DispTechsSet(self):
        return list(compress(list(range(1+1,len(self.Technologies[0].columns)+2)), list(self.Technologies[0].loc["Area (m2)"]==0)))

    
    def partloadtechs(self):
        PartLoad=self.Technologies[0].loc["MinLoad (%)",]/100

        partload=self.TechOutputs.iloc[0:1].mul(list(PartLoad),axis=1)
        partload=pd.concat([partload,self.TechOutputs.iloc[1:].mul(list(PartLoad),axis=1)], axis=0)
        partload=partload.abs()
        partload=partload.transpose()
        partload.index=list(range(1+1,len(self.TechOutputs.columns)+2))
        partload.columns=list(range(1,len(self.TechOutputs.index)+1))
        SolartechsSets=list(compress(list(range(1+1,len(self.Technologies[0].columns)+2)), list(self.Technologies[0].loc["Area (m2)"]>0)))

        for i in SolartechsSets:
            partload.drop(i, inplace=True)
            
        return list(partload.loc[partload.sum(axis=1)>0].index)
    
    
    #----------------------------------------------------------------------
    
    def LinearCost(self):
        dummy = {}
        for n in range(self.numberofhubs):
            LinearCost=self.Technologies[n].loc["CapCost (chf/kW)",]

            linCost=self.TechOutputs.iloc[0:1].mul(list(LinearCost),axis=1)
            linCost=pd.concat([linCost,self.TechOutputs.iloc[1:].mul(list(LinearCost),axis=1)], axis=0)

            linCost=linCost.transpose()
            for name in linCost.columns[1:]:
                linCost.loc[linCost["Electricity"] >1, name] = 0

            linCost.loc[linCost["Electricity"] <0, "Electricity"]=0
            linCost=linCost.abs()
            linCost.columns = self.DemandData[n].dropna(axis=1).columns

            addGrid=np.zeros(len(self.DemandData[0].dropna(axis=1).columns),)
            addGrid[0]=1 #add electricity to coupling matrix
            Grid=pd.DataFrame(addGrid,columns=["Grid"],index=self.DemandData[n].dropna(axis=1).columns).transpose()

            linCost=pd.concat([Grid,linCost])

            linCost.index=list(range(1,len(self.TechOutputs.columns)+2))
            linCost.columns=list(range(1,len(self.TechOutputs.index)+1))
            linCost.loc[1]=0
            
            dummy[n] = linCost.transpose()
        
        linCost = pd.Panel(dummy)
        linCapCosts={}
        linCapCosts=self.DictPanel(linCapCosts,linCost)
        
        return linCapCosts
     
    
    
    
    #----------------------------------------------------------------------
    ### Find which is the primary input for capacity #####
    def DisDemands(self):
        
        CHPlist=self.CHP_list()
        dispatch_demands=np.zeros((len(CHPlist), 2), dtype=int)

        for n,val in enumerate(CHPlist):
            counter=0
            for i, value in enumerate(np.array(self.TechOutputs[[val-2]],dtype=int)):
                if value[0]>0 and counter==0:
                    dispatch_demands[n,0]=i+1
                    counter=1
                if value[0]>0 and counter==1:
                    dispatch_demands[n,1]=i+1

        return dispatch_demands    
    
    
    
    #----------------------------------------------------------------------
    
    def InterestRate(self):
        Interest_rate=pd.read_excel(self.path,sheetname=self.GeneralSheet, skiprows=8, index_col=0, skip_footer=7) #
        Interest_rate=Interest_rate.dropna(axis=1,how='all')
        Interest_rate_R=Interest_rate.loc["Interest Rate r"][0]
        return Interest_rate_R
    
    
    
    
    #----------------------------------------------------------------------
    
    def LifeTime(self):
        dummy = {}
        for n in range(self.numberofhubs):
            Life=pd.DataFrame(list(self.Technologies[0].loc["Lifetime (yr)"]))
            Life.columns=[1]
            Life.index=list(range(2,len(self.TechOutputs.columns)+2))
            dummy[n]=Life
            
        Life = pd.Panel(dummy)
        
        Life= Life.to_frame()
        Life.reset_index(inplace = True)
        Life.index = Life['major']
        del Life['major']
        del Life['minor']
        Life.columns = [int(x)+1 for x in Life.columns]
        del Life.index.name
        Life = Life.transpose()

        lifeTimeTechs={}
        lifeTimeTechs=self.DictND(lifeTimeTechs, Life)
        return lifeTimeTechs
        
    
    
    
    #----------------------------------------------------------------------
        
    def NPV(self):
        Interest_rate_R=self.InterestRate()
        Life=pd.DataFrame(list(self.Technologies[0].loc["Lifetime (yr)"]))
        Life.columns=[1]
        Life.index=list(range(2,len(self.TechOutputs.columns)+2))
        
        NetPresentValue=1 / (((1 + Interest_rate_R) ** Life - 1) / (Interest_rate_R * ((1 + Interest_rate_R) ** Life)))
        NetPresentValueTech={}
        NetPresentValueTech=self.Dict1D_val_index(NetPresentValueTech,NetPresentValue)
        return NetPresentValueTech
        
    
    
    
    #----------------------------------------------------------------------
    
    def VarMaintCost(self):
        dummy = {}
        for n in range(self.numberofhubs):
            VarOMF=pd.DataFrame(list(self.Technologies[n].loc["OMVCost (chf/kWh)"]))
            VarOMF.columns=[1]
            VarOMF.index=list(range(1+1,len(self.TechOutputs.columns)+2))
            VarOMF.loc[1]=0
            dummy[n] = VarOMF
            
        VarOMF = pd.Panel(dummy)
        
        VarOMF= VarOMF.to_frame()
        VarOMF.reset_index(inplace = True)
        VarOMF.index = VarOMF['major']
        del VarOMF['major']
        del VarOMF['minor']
        VarOMF.columns = [int(x)+1 for x in VarOMF.columns]
        del VarOMF.index.name
        VarOMF = VarOMF.transpose()
        
        omvCosts={}
        omvCosts=self.DictND(omvCosts, VarOMF)
        return omvCosts
        
    
    
    
    #----------------------------------------------------------------------
    
    def CarbFactors(self):
        Carbon=pd.read_excel(self.path,sheetname=self.TechSheet, skiprows=24, index_col=0, skip_footer=16) #
        Carbon=Carbon.dropna(axis=0,how='all')
        Carbon=Carbon.dropna(axis=1,how='all')
        Carbon.index=[1]

        ElectricityCF=pd.read_excel(self.path,sheetname=self.GeneralSheet, skiprows=1, index_col=0, skip_footer=14) #
        ElectricityCF=ElectricityCF.dropna(axis=0,how='all')
        ElectricityCF=ElectricityCF.dropna(axis=1,how='all')
        del ElectricityCF["Price (chf/kWh)"]
        ElectricityCF.columns=[1]

        CarbonFactors= pd.concat([ElectricityCF.iloc[0], Carbon], axis=1)
        CarbonFactors.columns=list(range(1,len(CarbonFactors.columns)+1))
        CarbonFactors=CarbonFactors.transpose()

        carbonFactors={}
        carbonFactors=self.Dict1D(carbonFactors,CarbonFactors)
        
        return carbonFactors
       
    
    
    
    #----------------------------------------------------------------------
    
    def FuelPrice(self):
        Fuel=pd.read_excel(self.path,sheetname=self.GeneralSheet, skiprows=1, index_col=0, skip_footer=10) #
        Fuel=Fuel.dropna(axis=0,how='all')

        Carbon=pd.read_excel(self.path,sheetname=self.TechSheet, skiprows=24, index_col=0, skip_footer=16) #
        Carbon=Carbon.dropna(axis=0,how='all')
        Carbon=Carbon.dropna(axis=1,how='all')
        Carbon.index=[1]

        ElectricityCF=pd.read_excel(self.path,sheetname=self.GeneralSheet, skiprows=1, index_col=0, skip_footer=14) #
        ElectricityCF=ElectricityCF.dropna(axis=0,how='all')
        ElectricityCF=ElectricityCF.dropna(axis=1,how='all')
        del ElectricityCF["Price (chf/kWh)"]
        ElectricityCF.columns=[1]

        CarbonFactors= pd.concat([ElectricityCF.iloc[0], Carbon], axis=1)
        CarbonFactors.columns=list(range(1,len(CarbonFactors.columns)+1))
        CarbonFactors=CarbonFactors.transpose()

    
        for n,val in enumerate(Fuel["CO2 (kg/kWh)"]):
            for index,value in CarbonFactors.iterrows():
                if float(val)==float(value):
                    CarbonFactors.loc[index]=float(Fuel["Price (chf/kWh)"][n])

        opPrices={}
        opPrices=self.Dict1D(opPrices,CarbonFactors)
        
        return opPrices
     
    
    
    
    #----------------------------------------------------------------------
    
    def FeedIn(self):
        Tariff=pd.read_excel(self.path,sheetname=self.GeneralSheet, skiprows=11, index_col=0, skip_footer=1) #
        Tariff=Tariff.dropna(axis=0,how='all')
        Tariff=Tariff.dropna(axis=1,how='all')
        Tariff.columns=[1]
        Tariff.index=list(range(1,len(self.DemandData[0].dropna(axis=1).columns)+1))

        feedInTariffs={}
        feedInTariffs=self.Dict1D_val_index(feedInTariffs,Tariff)
        return feedInTariffs
    
    
    
    #### Storage ####
    #----------------------------------------------------------------------
    
    def StorageCh(self):
        maxStorCh={}
        MaxCharge=pd.DataFrame(list(self.StorageData.loc["max_charge"]))
        MaxCharge.columns=[1]
        MaxCharge = MaxCharge.transpose()
        MaxCharge.columns = [int(x)+1 for x in MaxCharge.columns]
        for i in range(2,self.numberofhubs+1):
            MaxCharge.loc[i] = MaxCharge.iloc[0]
        maxStorCh=self.DictND(maxStorCh, MaxCharge)
        
        return maxStorCh
    
    
    def StorageDisch(self):
        maxStorDisch={}
        MaxDischarge=pd.DataFrame(list(self.StorageData.loc["max_discharge"]))
        MaxDischarge.columns=[1]
        MaxDischarge = MaxDischarge.transpose()
        MaxDischarge.columns = [int(x)+1 for x in MaxDischarge.columns]
        for i in range(2,self.numberofhubs+1):
            MaxDischarge.loc[i] = MaxDischarge.iloc[0]
        maxStorDisch=self.DictND(maxStorDisch, MaxDischarge)
        return maxStorDisch
    
    
    def StorageLoss(self):
        lossesStorStanding={}
        losses=pd.DataFrame(list(self.StorageData.loc["decay"]))
        losses.columns=[1]
        losses = losses.transpose()
        losses.columns = [int(x)+1 for x in losses.columns]
        for i in range(2,self.numberofhubs+1):
            losses.loc[i] = losses.iloc[0]
        lossesStorStanding=self.DictND(lossesStorStanding, losses)
        return lossesStorStanding
    
    
    def StorageEfCh(self):
        chargingEff={}
        Ch_eff=pd.DataFrame(list(self.StorageData.loc["ch_eff"]))
        Ch_eff.columns=[1]
        Ch_eff = Ch_eff.transpose()
        Ch_eff.columns = [int(x)+1 for x in Ch_eff.columns]
        for i in range(2,self.numberofhubs+1):
            Ch_eff.loc[i] = Ch_eff.iloc[0]
        chargingEff=self.DictND(chargingEff, Ch_eff)
        return chargingEff
    
    def StorageEfDisch(self):
        dischLosses={}
        Disch_eff=pd.DataFrame(list(self.StorageData.loc["disch_eff"]))
        Disch_eff.columns=[1]
        Disch_eff = Disch_eff.transpose()
        Disch_eff.columns = [int(x)+1 for x in Disch_eff.columns]
        for i in range(2,self.numberofhubs+1):
            Disch_eff.loc[i] = Disch_eff.iloc[0]
        dischLosses=self.DictND(dischLosses, Disch_eff)
        return dischLosses
    
    
    def StorageMinSoC(self):
        minSoC={}
        min_state=pd.DataFrame(list(self.StorageData.loc["min_state"]))
        min_state.columns=[1]
        min_state = min_state.transpose()
        min_state.columns = [int(x)+1 for x in min_state.columns]
        for i in range(2,self.numberofhubs+1):
            min_state.loc[i] = min_state.iloc[0]
        minSoC=self.DictND(minSoC, min_state)
        return minSoC
    
    
    def StorageLife(self):
        lifeTimeStorages={}
        LifeBattery=pd.DataFrame(list(self.StorageData.loc["LifeBat (year)"]))
        LifeBattery.columns=[1]
        LifeBattery = LifeBattery.transpose()
        LifeBattery.columns = [int(x)+1 for x in LifeBattery.columns]
        for i in range(2,self.numberofhubs+1):
            LifeBattery.loc[i] = LifeBattery.iloc[0]
        lifeTimeStorages=self.DictND(lifeTimeStorages, LifeBattery)
        return lifeTimeStorages
    
    
    def StorageLinCost(self):
        linStorageCosts={}
        LinearCostStorage=pd.DataFrame(list(self.StorageData.loc["CostBat (chf/kWh)"]))
        LinearCostStorage.columns=[1]
        LinearCostStorage = LinearCostStorage.transpose()
        LinearCostStorage.columns = [int(x)+1 for x in LinearCostStorage.columns]
        for i in range(2,self.numberofhubs+1):
            LinearCostStorage.loc[i] = LinearCostStorage.iloc[0]
        linStorageCosts=self.DictND(linStorageCosts, LinearCostStorage)
        return linStorageCosts
    
    
    def StorageNPV(self):
        Interest_rate_R=self.InterestRate()
        LifeBattery=pd.DataFrame(list(self.StorageData.loc["LifeBat (year)"]))
        LifeBattery.columns=[1]
        NetPresentValueStor={}
        NetPresentValueStorage=1 / (((1 + Interest_rate_R) ** LifeBattery - 1) / (Interest_rate_R * ((1 + Interest_rate_R) ** LifeBattery)))
        '''
        NetPresentValueStorage.columns = [1]
        NetPresentValueStorage = NetPresentValueStorage.transpose()
        NetPresentValueStorage.columns = [int(x)+1 for x in NetPresentValueStorage.columns]
        for i in range(2,self.numberofhubs+1):
            NetPresentValueStorage.loc[i] = NetPresentValueStorage.iloc[0]
        NetPresentValueStor=self.DictND(NetPresentValueStor,NetPresentValueStorage)
        return NetPresentValueStor
        '''
        NetPresentValueStor=self.Dict1D(NetPresentValueStor,NetPresentValueStorage)
        return NetPresentValueStor
    
    def StorageBinary(self):
        dummy = np.zeros((self.numberofhubs, self.numberofdemands, ))
        for i in range(self.numberofdemands):
            for j in range(self.numberofhubs):
                if str(j+1) in list(self.StorageData.loc["Hubs"][i]):
                    dummy[j,i] = 1

        StorageY= pd.DataFrame(dummy)
        StorageY.columns= [int(x)+1 for x in StorageY.columns]
        StorageY.index = [int(x)+1 for x in StorageY.index]

        StorageYbinary={}
        StorageYbinary = self.DictND(StorageYbinary, StorageY)
        return StorageYbinary

     

In [303]:
excel_path=r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\data\General_input_MultiV2.xlsx'
data=InputData(excel_path)


In [304]:
#-----------------------------------------------------------------------------#
## Creating a model ##
#-----------------------------------------------------------------------------#

model = ConcreteModel()

# sets definition
model.Time = RangeSet(1, data.DemandData.shape[1]) #0 is items, 1 is row
model.SubTime = RangeSet(2, data.DemandData.shape[1], within=model.Time) #0 is items, 1 is row
model.In = RangeSet(1, data.Technologies.shape[2]+1) #0 is items, 1 is row , 2 is column , it is assumed that the layout of hub's technologies is the same and the choice of technology is controled by max cap in each DF
#model.W_Ogrid = Set(initialize=list(range(1+1,len(data.Technologies.columns)+2)), within=model.In) #techs w/o grid


model.Out = RangeSet(1, data.numberofdemands) #0 is items, 1 is row , 2 is column
number_of_demands= list(range(1, data.numberofdemands+1))

model.SolarTechs = Set(initialize=data.SolarSet(), within=model.In)

model.DispTechs = Set(initialize=data.DispTechsSet(), within=model.In)
model.Techs = Set(initialize=list(range(2,data.Technologies[0].shape[1]+2)), within=model.In)
model.PartLoad=Set(initialize=data.partloadtechs(), within=model.In)

model.CHP = Set(initialize=data.CHP_list(), within=model.In) # set dispatch tech set
model.roof_tech = Set(initialize=data.Roof_tech(), within=model.In) # set tech with roof area set
#model.gas_tech = Set(initialize=[3,4,7,8], within=model.In) #

numberofhubs=data.DemandData.shape[0]
model.hubs=RangeSet(1,data.DemandData.shape[0]) #0 is items /number of hubs
model.hub_i=RangeSet(1,data.DemandData.shape[0], within=model.hubs)
#model.hub_j=Set(within=model.hubs)
model.hub_j=RangeSet(1,data.DemandData.shape[0], within=model.hubs)


# coupling matrix & Technical parameters
model.cMatrix = Param(model.In, model.Out, initialize=data.cMatrix())                                      # coupling matrix technology efficiencies 
model.maxCapTechs = Param(model.hubs, model.DispTechs, initialize=data.MaxCapacity())
model.maxCapTechsAll = Param(model.hubs, model.Techs, initialize=data.MaxCapacityALL())
model.maxStorCh = Param(model.hubs, model.Out, initialize=data.StorageCh())
model.maxStorDisch = Param(model.hubs, model.Out, initialize= data.StorageDisch())

model.lossesStorStanding = Param(model.hubs, model.Out, initialize = data.StorageLoss())
model.chargingEff = Param(model.hubs, model.Out, initialize = data.StorageEfCh())
model.dischLosses = Param(model.hubs, model.Out, initialize = data.StorageEfDisch())
model.minSoC = Param(model.hubs, model.Out, initialize = data.StorageMinSoC())
model.partLoad = Param(model.In, model.Out, initialize=data.PartLoad()) #PartloadInput
model.maxSolarArea = Param(initialize=500)

# carbon factors
model.carbonFactors = Param(model.In, initialize=data.CarbFactors())
model.maxCarbon = Param(initialize=650000)

# Cost parameters
model.linCapCosts = Param(model.hubs, model.In, model.Out, initialize= data.LinearCost())         # Technologies capital costs
model.linStorageCosts = Param(model.hubs, model.Out, initialize = data.StorageLinCost())
model.opPrices = Param( model.In, initialize=data.FuelPrice())   #model.hubs,                    # Operating prices technologies
model.feedInTariffs = Param(model.Out, initialize=data.FeedIn())               # feed-in tariffs
model.omvCosts = Param(model.hubs, model.In, initialize=data.VarMaintCost())                            # Maintenance costs
model.interestRate = Param(within=NonNegativeReals, initialize=data.InterestRate())
# lifetime
model.lifeTimeTechs = Param(model.hubs, model.In, initialize = data.LifeTime())
model.lifeTimeStorages = Param(model.hubs, model.Out, initialize = data.StorageLife())

## DH parameters
model.HeatLosses=Param(model.hub_i, model.hub_j, initialize=0.001)
#model.Distance=Param(model.hub_i, model.hub_j, initialize=)
model.PipeCost=Param(initialize=200)
model.Losses=Param(model.hub_i, model.hub_j, initialize=0.001 )


## Declaring Global Parameters ##
model.timeHorizon = Param(within=NonNegativeReals, initialize=20)
model.bigM = Param(within=NonNegativeReals, initialize=5000)

#loads

model.loads = Param(model.hub_i, model.Time, model.Out, initialize=data.Demands())
model.solarEm = Param( model.Time, initialize=data.SolarData())


## Global variables
model.P = Var(model.hubs, model.Time, model.In, domain=NonNegativeReals)
model.Pexport = Var(model.hubs, model.Time, model.Out, domain=NonNegativeReals)
model.Capacities = Var(model.hubs, model.In, model.Out, domain=NonNegativeReals)
model.Ytechnologies = Var(model.hubs, model.In, model.Out, domain=Binary)
model.Yon = Var(model.hubs, model.Time, model.In, domain=Binary)
model.TotalCost = Var(domain=Reals)
model.OpCost = Var(domain=NonNegativeReals)
model.MaintCost = Var(domain=NonNegativeReals)
model.IncomeExp = Var(domain=NonNegativeReals)
model.InvCost = Var(domain=NonNegativeReals)
model.TotalCarbon = Var(domain=Reals)
model.TotalCarbon2 = Var(domain=Reals)
model.NetPresentValueTech = Param(model.In, domain=NonNegativeReals, initialize=data.NPV())
model.NetPresentValueStor = Param(model.Out, domain=NonNegativeReals, initialize=data.StorageNPV())

## DH variables
model.DH_Q = Var(model.hub_i, model.hub_j,model.Time, model.Out, domain=NonNegativeReals)
model.Ypipeline = Var(model.hub_i, model.hub_j, domain=Binary)

                      
#Storage variables
model.Qin = Var(model.hubs, model.Time, model.Out, domain=NonNegativeReals)
model.Qout = Var(model.hubs, model.Time, model.Out, domain=NonNegativeReals)
model.E = Var(model.hubs, model.Time, model.Out, domain=NonNegativeReals)
model.StorageCap = Var(model.hubs, model.Out, domain=NonNegativeReals)
#model.Ystorage = Var(model.hubs, model.Out, domain=Binary)
model.Ystorage = Param(model.hubs, model.Out, domain=Binary, initialize=data.StorageBinary())
#model.maxStorageCap = Param(model.Out, initialize= maxStorageCap)

In [60]:
model.capacityConst.display()

capacityConst : Size=2700
ERROR: evaluating expression: No value for uninitialized NumericValue object P[5,16,8]
	    (expression: 0.519*P[5,16,8] - Capacities[5,8,2])
ERROR: evaluating expression: No value for uninitialized NumericValue object P[4,13,4]
	    (expression: 0.3*P[4,13,4] - Capacities[4,4,1])
ERROR: evaluating expression: No value for uninitialized NumericValue object P[2,29,8]
	    (expression: 0.519*P[2,29,8] - Capacities[2,8,2])
ERROR: evaluating expression: No value for uninitialized NumericValue object P[3,20,7]
	    (expression: 0.3*P[3,20,7] - Capacities[3,7,1])
ERROR: evaluating expression: No value for uninitialized NumericValue object P[5,19,4]
	    (expression: 0.519*P[5,19,4] - Capacities[5,4,2])
ERROR: evaluating expression: No value for uninitialized NumericValue object P[1,38,5]
	    (expression: 0.14*P[1,38,5] - Capacities[1,5,1])
ERROR: evaluating expression: No value for uninitialized NumericValue object P[4,25,8]
	    (expression: 0.3*P[4,25,8] - Capaci

TypeError: not all arguments converted during string formatting

In [305]:
#-----------------------------------------------------------------------------#
## GLobal constraints
#-----------------------------------------------------------------------------#
def MultipleloadsBalance_rule(model, i,  t, out):
    return (model.loads[i, t,out] + model.Pexport[i, t,out] <= ((model.Qout[ i, t,out] - model.Qin[i, t,out] + 
                                                        sum(model.P[i, t,inp]*model.cMatrix[inp,out] for inp in model.In) 
                                                                + sum(model.DH_Q[j,i,t,out]*(1-model.Losses[i,j])- model.DH_Q[i,j,t,out] for j in model.hub_i ))))

def loadsBalance_rule(model, i, t, out):
    return (model.loads[i, t,out] + model.Pexport[i, t,out] <= (model.Qout[ i, t,out] - model.Qin[i, t,out] + 
                                                        sum(model.P[i, t,inp]*model.cMatrix[inp,out] for inp in model.In)))

if numberofhubs>1:
    model.loadsBalance = Constraint(model.hub_i,  model.Time, model.Out, rule=MultipleloadsBalance_rule)
    '''
    model.DHcon = ConstraintList()
    for i in range(1, numberofhubs+1):
        for t in range(1,data.DemandData.shape[1]+1):
            for out in range(1,data.numberofdemands+1):
                gen = (j for j in model.hub_i if j!=i)
                model.DHcon.add((model.loads[i, t,out] + model.Pexport[i, t,out]) <= ((model.Qout[ i, t,out] - model.Qin[i, t,out] + 
                                                        sum(model.P[i, t,inp]*model.cMatrix[inp,out] for inp in model.In) 
                                                        + sum(model.DH_Q[j,i,t,out]*model.Losses[i,j]- model.DH_Q[i,j,t,out] for j in gen))))

    '''
    
    
else:
    model.loadsBalance = Constraint(model.hub_i, model.Time, model.Out, rule=loadsBalance_rule)

def capacityConst_rule(model, i, t, inp, out):
    if model.cMatrix[inp,out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.P[i, t, inp]  * model.cMatrix[inp,out]<= model.Capacities[i, inp,out])
model.capacityConst = Constraint(model.hub_i, model.Time, model.In, model.Out, rule=capacityConst_rule)


def maxCapacity_rule2(model, i, tech, out):
    return (model.Capacities[i, tech, out] <= model.maxCapTechsAll[i, tech])
model.maxCapacity2 = Constraint(model.hub_i, model.Techs, model.Out, rule=maxCapacity_rule2)


'''
def carbonConst_rule(model):
    return (model.TotalCarbon <= model.maxCarbon)
model.carbonConst = Constraint(rule=carbonConst_rule)
'''

'\ndef carbonConst_rule(model):\n    return (model.TotalCarbon <= model.maxCarbon)\nmodel.carbonConst = Constraint(rule=carbonConst_rule)\n'

In [306]:
#-----------------------------------------------------------------------------#
## Specific constraints 
#-----------------------------------------------------------------------------#

def partLoadL_rule(model, i, t, disp, out):  
    if model.cMatrix[disp,out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.partLoad[disp, out] * model.Capacities[i, disp, out] <= 
                                                                    (model.P[i, disp, out] * model.cMatrix[disp,out] 
                                                                    + model.bigM * (1 - model.Yon[i, t, disp])))
model.partLoadL = Constraint(model.hub_i, model.Time, model.PartLoad, model.Out, rule=partLoadL_rule)

def partLoadU_rule(model, i, t, disp, out):    
    if model.cMatrix[disp,out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.P[i, t, disp] * model.cMatrix[disp, out] <= model.bigM * model.Yon[i, t, disp])
model.partLoadU = Constraint(model.hub_i, model.Time, model.PartLoad, model.Out, rule=partLoadU_rule)



def solarInput_rule(model, i, t, sol, out):
    if model.cMatrix[sol, out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.P[i, t, sol] == model.solarEm[t] * model.Capacities[i, sol, out])
model.solarInput = Constraint(model.hub_i, model.Time, model.SolarTechs, model.Out, rule=solarInput_rule) 


def roofArea_rule(model,i, roof, demand):
    return (sum(model.Capacities[i, roof,d] for d in number_of_demands) <= model.maxSolarArea) #DemandData.shape[1]
model.roofArea = Constraint(model.hub_i, model.roof_tech,model.Out,rule=roofArea_rule) #model.roof_tech

def fixCostConst_rule(model, i, inp, out):
    return (model.Capacities[i, inp,out] <= model.bigM * model.Ytechnologies[i, inp,out])
model.fixCostConst = Constraint(model.hub_i, model.In, model.Out, rule=fixCostConst_rule)

dispatch_demands=data.DisDemands() 
CHP_list=data.CHP_list()
model.con = ConstraintList()
for i in range(1, numberofhubs+1):
    for x in range(1, len(dispatch_demands)+1):
        model.con.add(model.Capacities[i, CHP_list[x-1],dispatch_demands[x-1,1]] == model.cMatrix[CHP_list[x-1],dispatch_demands[x-1,1]] / model.cMatrix[CHP_list[x-1],dispatch_demands[x-1,0]] * model.Capacities[i, CHP_list[x-1],dispatch_demands[x-1,0]])
        model.con.add(model.Ytechnologies[i, CHP_list[x-1],dispatch_demands[x-1,0]]==model.Ytechnologies[i, CHP_list[x-1],dispatch_demands[x-1,1]])
        model.con.add(model.Capacities[i, CHP_list[x-1],dispatch_demands[x-1,0]] <= model.maxCapTechs[i, CHP_list[x-1]] * model.Ytechnologies[i, CHP_list[x-1],dispatch_demands[x-1,0]])

#model.con.construct()



def DH_rule(model, i, j, t, out):
    return (model.DH_Q[i,j,t,out] <= model.Ypipeline[i,j] * model.bigM)

if numberofhubs>1:
    model.DH_network = Constraint(model.hub_i, model.hub_j, model.Time, model.Out, rule=DH_rule)
    
#def DHy_rule(model, i, j):
#    return (model.Ypipeline[i,j]+model.Ypipeline[j,i] <= 1)

if numberofhubs>1:
    #model.DH_network = Constraint(model.hub_i, model.hub_j,  rule=DHy_rule)
    model.DHYcon = ConstraintList()
    for i in range(1, numberofhubs+1-1):
        model.DHYcon.add((model.Ypipeline[i,i]<= 0 ))
        for j in range(i+1, numberofhubs+1):
                model.DHYcon.add((model.Ypipeline[j,j]<= 0 ))
                model.DHYcon.add((model.Ypipeline[i,j]+model.Ypipeline[j,i] <= 1 ))
            #else:
                #model.DHYcon.add((model.Ypipeline[i,j]+model.Ypipeline[j,i] == 0 ))
                

                

In [307]:
#-----------------------------------------------------------------------------#
## Storage constraints
#-----------------------------------------------------------------------------#

def storageBalance_rule(model, i, t, out):
    return (model.E[i, t, out] == (model.lossesStorStanding[i,out] * model.E[i, (t-1), out] 
                                + model.chargingEff[i,out] * model.Qin[i, t, out] 
                                - (1/model.dischLosses[i,out]) * model.Qout[i, t, out]))
model.storageBalance = Constraint(model.hub_i, model.SubTime, model.Out, rule=storageBalance_rule)
'''
model.StorCon = ConstraintList()
for x in range(1, DemandData.shape[1]+1):
    
    #model.StorCon.add(model.E[1, x] == model.StorageCap[x] * model.minSoC[x])
    model.StorCon.add(model.E[1, x] == model.E[8760, x])
    #model.StorCon.add(model.Qout[1, x] == 0)
'''    

'''
def storageInitBattery_rule(model):
    return (model.E[1, 1] == model.StorageCap[1] * model.minSoC[1])
model.storageInitBattery = Constraint(rule=storageInitBattery_rule)

def storageInitThermal1_rule(model):
    return (model.E[1, 2] == model.E[8760, 2])
model.storageInitThermal1 = Constraint(rule=storageInitThermal1_rule)

def storageInitThermal2_rule(model):
    return (model.Qout[1, 2] == 0)
model.storageInitThermal2 = Constraint(rule=storageInitThermal2_rule)
'''


def storageChargeRate_rule(model, i, t, out):
    return (model.Qin[i, t,out] <= model.maxStorCh[i, out] * model.StorageCap[i, out])
model.storageChargeRate = Constraint(model.hub_i, model.Time, model.Out, rule=storageChargeRate_rule)

def storageDischRate_rule(model, i, t, out):
    return (model.Qout[i, t,out] <= model.maxStorDisch[i,out] * model.StorageCap[i, out])
model.storageDischRate = Constraint(model.hub_i,model.Time, model.Out, rule=storageDischRate_rule)
'''
def storageMinState_rule(model, t, out):
    return (model.E[t,out] >= model.StorageCap[out] * model.minSoC[out])
model.storageMinState = Constraint(model.Time, model.Out, rule=storageMinState_rule)
'''

def storageCap_rule(model, i, t, out):
    return (model.E[i, t,out] <= model.StorageCap[i, out] )
model.storageCap = Constraint(model.hub_i,model.Time, model.Out, rule=storageCap_rule)

def storageMaxCap_rule(model, i,  out):
    return (model.StorageCap[i, out] <= model.bigM * model.Ystorage[i,out] )
model.storageMaxCap = Constraint(model.hub_i,  model.Out, rule=storageMaxCap_rule)


In [308]:
#-----------------------------------------------------------------------------#
## Objective functions
#-----------------------------------------------------------------------------#
def objective_rule(model):
    return (model.TotalCost)
model.Total_Cost = Objective(rule=objective_rule, sense=minimize)

#def objective_rule(model):
#    return (model.TotalCarbon2)
#model.Total_Carbon = Objective(rule=objective_rule, sense=minimize)

def opCost_rule(model):
    return(model.OpCost == ((sum (model.opPrices[inp] 
                            * sum(model.P[i, t,inp] for i in model.hub_i for t in model.Time)
                            for inp in model.In)))
                            )
model.opCost = Constraint(rule=opCost_rule) 

def maintenanceCost_rule(model):
    return(model.MaintCost == (sum(model.P[i, t,inp] * 
                              model.cMatrix[inp,out] * model.omvCosts[i, inp]
                              for i in model.hub_i for t in model.Time for inp in model.In for out in model.Out)
                              ))
model.maintCost = Constraint(rule=maintenanceCost_rule)

def incomeExp_rule(model):
    return(model.IncomeExp == (sum(model.feedInTariffs[out] * 
                            sum(model.Pexport[i, t,out] for i in model.hub_i for t in model.Time) 
                            for out in model.Out))
                            )
model.incomeExp = Constraint(rule=incomeExp_rule)

def DHinvCost_rule(model):
    return(model.InvCost == (sum(model.NetPresentValueTech[inp] * (model.linCapCosts[i, inp,out] * model.Capacities[i, inp,out] ) for i in model.hub_i for inp in model.Techs for out in model.Out) #+ model.fixCapCosts[inp,out] * model.Ytechnologies[inp,out]
                            + sum(model.NetPresentValueStor[out] * model.linStorageCosts[i, out] * model.StorageCap[i, out] for i in model.hub_i for out in model.Out)))
                               #+ sum(model.Distance[i,j]*model.PipeCost*model.Ypipeline[i,j] for i in model.hub_i for j in model.hub_j))

def invCost_rule(model):
    return(model.InvCost == (sum(model.NetPresentValueTech[inp] * (model.linCapCosts[i, inp,out] * model.Capacities[i, inp,out] ) for i in model.hub_i for inp in model.Techs for out in model.Out) #+ model.fixCapCosts[inp,out] * model.Ytechnologies[inp,out]
                            + sum(model.NetPresentValueStor[out] * model.linStorageCosts[i, out] * model.StorageCap[i, out] for i in model.hub_i for out in model.Out)
                            ))

if numberofhubs>1:
    model.invCost = Constraint(rule=DHinvCost_rule) 
else:
    model.invCost = Constraint(rule=invCost_rule) 

def totalCost_rule(model):
    return(model.TotalCost == model.InvCost + model.OpCost + model.MaintCost - model.IncomeExp)
model.totalCost = Constraint(rule=totalCost_rule) 

def totalCarbon2_rule(model):
    return(model.TotalCarbon2 == model.TotalCarbon)
model.totalCarbon2 = Constraint(rule=totalCarbon2_rule)

def totalCarbon_rule(model):
    return(model.TotalCarbon == sum(model.carbonFactors[inp] * sum(model.P[i, t,inp] for i in model.hub_i for t in model.Time) for inp in model.In))
model.totalCarbon = Constraint(rule=totalCarbon_rule)


In [309]:
#-----------------------------------------------------------------------------#
## Print model ##
#-----------------------------------------------------------------------------#
#model.pprint()

#instance = model.create(r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\generic_model.dat')


opt = SolverFactory("gurobi")
opt.options["mipgap"]=0.05
opt.options["FeasibilityTol"]=1e-05
solver_manager = SolverManagerFactory("serial")
#results = solver_manager.solve(instance, opt=opt, tee=True,timelimit=None, mipgap=0.1)

results = solver_manager.solve(model, opt=opt, tee=True,timelimit=None)

#sends results to stdout
#results.write()


def pyomo_save_results(options=None, instance=None, results=None):
    OUTPUT = open(r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\data\Results_generic_hub.txt','w')
    print(results, file=OUTPUT)
    OUTPUT.close()
#pyomo_save_results(instance=instance, results=results)    

#Example of how to print variables
#print(instance.TotalCost.value)
print(model.TotalCost.value)




#for i in instance.Pnaturalgas:
#    print (instance.Pnaturalgas[i], instance.Pnaturalgas[i].value)

#print("--- %s seconds ---" % (time.time() - start_time))

## create the instance
#instance = model.create(’DiseaseEstimation.dat’)
## define the solver and its options
#solver = ’ipopt’
#opt = SolverFactory( solver )
#if opt is None:
#raise ValueError, "Problem constructing solver \
#‘"+str(solver)
#opt.set_options(’max_iter=2’)
## create the solver manager
#solver_manager = SolverManagerFactory( ’serial’ )
## solve
#results = solver_manager.solve(instance, opt=opt, tee=True,\
#timelimit=None)
#instance.load(results)
## display results
#display(instance)



Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Parameter mipgap unchanged
   Value: 0.05  Min: 0.0  Max: 1e+100  Default: 0.0001
Changed value of parameter FeasibilityTol to 1e-05
   Prev: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Optimize a model with 12962 rows, 9093 columns and 43066 nonzeros
Variable types: 8258 continuous, 835 integer (835 binary)
Coefficient statistics:
  Matrix range     [1e-03, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Presolve removed 4364 rows and 2367 columns
Presolve time: 0.17s
Presolved: 8598 rows, 6726 columns, 27099 nonzeros
Variable types: 6176 continuous, 550 integer (550 binary)

Root relaxation: objective 1.728536e+04, 3446 iterations, 0.08 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1

In [227]:
print(model.TotalCarbon.value)

1859.59884795


In [293]:
model.P.get_values()

{(1, 1, 1): 3.0,
 (1, 1, 2): 0.0,
 (1, 1, 3): 22.3404255319,
 (1, 1, 4): 0.0,
 (1, 1, 5): 0.0,
 (1, 1, 6): 0.0,
 (1, 1, 7): 0.0,
 (1, 1, 8): 0.0,
 (1, 1, 9): 22.3404255319,
 (1, 2, 1): 3.0,
 (1, 2, 2): 0.0,
 (1, 2, 3): 25.5319148936,
 (1, 2, 4): 0.0,
 (1, 2, 5): 0.0,
 (1, 2, 6): 0.0,
 (1, 2, 7): 0.0,
 (1, 2, 8): 0.0,
 (1, 2, 9): 25.5319148936,
 (1, 3, 1): 3.0,
 (1, 3, 2): 0.0,
 (1, 3, 3): 27.6595744681,
 (1, 3, 4): 0.0,
 (1, 3, 5): 0.0,
 (1, 3, 6): 0.0,
 (1, 3, 7): 0.0,
 (1, 3, 8): 0.0,
 (1, 3, 9): 27.6595744681,
 (1, 4, 1): 3.0,
 (1, 4, 2): 0.0,
 (1, 4, 3): 29.7872340426,
 (1, 4, 4): 0.0,
 (1, 4, 5): 0.0,
 (1, 4, 6): 0.0,
 (1, 4, 7): 0.0,
 (1, 4, 8): 0.0,
 (1, 4, 9): 29.7872340426,
 (1, 5, 1): 3.0,
 (1, 5, 2): 0.0,
 (1, 5, 3): 31.914893617,
 (1, 5, 4): 0.0,
 (1, 5, 5): 0.0,
 (1, 5, 6): 0.0,
 (1, 5, 7): 0.0,
 (1, 5, 8): 0.0,
 (1, 5, 9): 31.914893617,
 (1, 6, 1): 3.0,
 (1, 6, 2): 0.0,
 (1, 6, 3): 32.9787234043,
 (1, 6, 4): 0.0,
 (1, 6, 5): 0.0,
 (1, 6, 6): 0.0,
 (1, 6, 7): 0.0,
 (1, 6, 

In [312]:
model.Capacities.get_values()

{(1, 1, 1): 5000.0,
 (1, 1, 2): 0.0,
 (1, 1, 3): 0.0,
 (1, 2, 1): 0.0,
 (1, 2, 2): 0.0,
 (1, 2, 3): 0.0,
 (1, 3, 1): 0.0,
 (1, 3, 2): 82.0,
 (1, 3, 3): 0.0,
 (1, 4, 1): 0.0,
 (1, 4, 2): 0.0,
 (1, 4, 3): 0.0,
 (1, 5, 1): 0.0,
 (1, 5, 2): 0.0,
 (1, 5, 3): 0.0,
 (1, 6, 1): 0.0,
 (1, 6, 2): 0.0,
 (1, 6, 3): 0.0,
 (1, 7, 1): 0.0,
 (1, 7, 2): 0.0,
 (1, 7, 3): 0.0,
 (1, 8, 1): 0.0,
 (1, 8, 2): 0.0,
 (1, 8, 3): 0.0,
 (1, 9, 1): 0.0,
 (1, 9, 2): 0.0,
 (1, 9, 3): 118.012012012,
 (2, 1, 1): 5000.0,
 (2, 1, 2): 0.0,
 (2, 1, 3): 0.0,
 (2, 2, 1): 0.0,
 (2, 2, 2): 0.0,
 (2, 2, 3): 0.0,
 (2, 3, 1): 0.0,
 (2, 3, 2): 82.0,
 (2, 3, 3): 0.0,
 (2, 4, 1): 0.0,
 (2, 4, 2): 0.0,
 (2, 4, 3): 0.0,
 (2, 5, 1): 0.0,
 (2, 5, 2): 0.0,
 (2, 5, 3): 0.0,
 (2, 6, 1): 0.0,
 (2, 6, 2): 0.0,
 (2, 6, 3): 0.0,
 (2, 7, 1): 0.0,
 (2, 7, 2): 0.0,
 (2, 7, 3): 0.0,
 (2, 8, 1): 0.0,
 (2, 8, 2): 0.0,
 (2, 8, 3): 0.0,
 (2, 9, 1): 0.0,
 (2, 9, 2): 0.0,
 (2, 9, 3): 82.0,
 (3, 1, 1): 5000.0,
 (3, 1, 2): 0.0,
 (3, 1, 3): 0.0,
 (3, 2, 1

In [311]:
model.DH_Q.get_values()

{(4, 5, 17, 3): 0.0,
 (2, 3, 29, 1): 0.0,
 (2, 4, 36, 2): 0.0,
 (2, 5, 17, 1): 0.0,
 (1, 2, 9, 1): 0.0,
 (4, 3, 16, 3): 0.0,
 (2, 1, 30, 2): 0.0,
 (5, 5, 43, 3): 0.0,
 (1, 5, 11, 3): 0.0,
 (5, 1, 4, 1): 0.0,
 (2, 2, 35, 1): 0.0,
 (1, 5, 32, 3): 0.0,
 (2, 4, 17, 2): 0.0,
 (3, 5, 21, 3): 0.0,
 (5, 2, 6, 2): 0.0,
 (1, 2, 36, 3): 0.0,
 (4, 3, 5, 2): 0.0,
 (2, 1, 19, 3): 0.0,
 (5, 5, 22, 2): 0.0,
 (4, 4, 28, 1): 0.0,
 (5, 1, 11, 2): 0.0,
 (1, 1, 8, 3): 0.0,
 (5, 4, 44, 3): 0.0,
 (1, 5, 21, 1): 0.0,
 (4, 5, 18, 2): 0.0,
 (3, 2, 17, 2): 0.0,
 (2, 4, 40, 1): 0.0,
 (3, 1, 5, 3): 0.0,
 (4, 2, 18, 3): 0.0,
 (3, 5, 24, 2): 0.0,
 (5, 2, 11, 3): 0.0,
 (4, 3, 40, 2): 0.0,
 (4, 3, 10, 1): 0.0,
 (3, 4, 22, 1): 0.0,
 (5, 1, 37, 3): 0.0,
 (2, 2, 9, 3): 0.0,
 (3, 3, 38, 3): 0.0,
 (1, 5, 10, 1): 0.0,
 (3, 2, 30, 1): 0.0,
 (2, 4, 39, 2): 0.0,
 (5, 3, 42, 1): 0.0,
 (2, 5, 13, 2): 0.0,
 (2, 1, 16, 2): 0.0,
 (5, 5, 21, 3): 0.0,
 (4, 3, 17, 1): 0.0,
 (1, 3, 43, 2): 0.0,
 (5, 1, 40, 2): 0.0,
 (2, 2, 28, 2): 0.0,

In [310]:
model.Ypipeline.get_values()

{(1, 1): 0.0,
 (1, 2): 1.0,
 (1, 3): 1.0,
 (1, 4): 1.0,
 (1, 5): 1.0,
 (2, 1): 0.0,
 (2, 2): 0.0,
 (2, 3): 1.0,
 (2, 4): 1.0,
 (2, 5): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.0,
 (3, 4): 1.0,
 (3, 5): 1.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 0.0,
 (4, 5): 0.0,
 (5, 1): 0.0,
 (5, 2): 1.0,
 (5, 3): 0.0,
 (5, 4): 1.0,
 (5, 5): 0.0}

In [20]:
P_matrix=np.zeros(shape=(DemandData.shape[0],Technologies.shape[1]+1))
for i in range(1,DemandData.shape[0]+1):
    for j in range(1,Technologies.shape[1]+1+1):
        P_matrix[i-1,j-1]=model.P[i,j].value

In [21]:
P=pd.DataFrame(P_matrix)